In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import resnet50
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
#from tensorflow.python.keras.applications.resnet50 import ResNet50

from sklearn.model_selection import train_test_split

In [2]:
def pre_process_data(X,Y):
    
    X_p = resnet50.preprocess_input(X)
    Y_p = tf.keras.util.to_category(Y,11)

In [3]:
from keras.models import Model, Sequential

In [4]:
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

In [5]:
def parse_data(x, y):
    x = x.decode()

    num_class = 11
    size = 224

    image = read_image(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return image, label

In [6]:
def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((11))
    return x, y

In [7]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [8]:
# model = resnet50.ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
# model.trainable = True
# x = model.output
# x = Flatten()(x)
# x=Dropout(0.35)(x)
# x=Dense(units=1000,activation='relu')(x)
# x=Dropout(0.4)(x)
# x=Dense(units=750,activation='relu')(x)
# x=Dropout(0.4)(x)
# x=Dense(units=1000,activation='relu')(x)
# x=Dropout(0.8)(x)
# #clf.add(Dense(units=120,activation='softmax')
# #stochastic gradient descent -Adam -optimizer
# #loss func categorical cross entropy
# #metrics = accuracy
# #clf.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# predictions = Dense(11, activation='softmax')(x)

In [9]:
#def build_model(size, num_classes):
    #inputs = tf.keras.Input(shape=(size, size, 3))
    #backbone = resnet50.ResNet50(weights='imagnet',include_top=False, input_shape=(224, 224, 3)
model = resnet50.ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
model.trainable = True
x = model.output
#     print(backbone.summary)
#     print(x.shape)
#     x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
#x = Dropout(0.2)(x)
#     x = Dense(1024, activation="relu")(x)
#     x = Dense(num_classes, activation="softmax")(x)
#    x = backbone.output
#     x = Flatten()(x)
x = Dropout(0.35)(x)
x = Dense(units=1000,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(units=750,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(units=1000,activation='relu')(x)
x = Dropout(0.8)(x)
    #clf.add(Dense(units=120,activation='softmax')
    #stochastic gradient descent -Adam -optimizer
    #loss func categorical cross entropy
    #metrics = accuracy
    #clf.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
predictions = Dense(11, activation='softmax')(x)
#model = Model(inputs, x)
#return model

In [10]:
import keras
main_model = Model(inputs=model.input, outputs=predictions)

#train only the hidden layers and output layer, donot train the resnet model
for curLayer in model.layers:
    curLayer.trainable = False
    
main_model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
main_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 125,453,473
Trainable params: 101,865,761
Non-trainable params: 23,587,712
__________________________________________________________________________________________________


In [11]:
path = "E:\DAG_BREED_CLASSIFICATION\Dog-Breed-Classifier-using-TF2.0-master/"
train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "labels.csv")
#     ids = np.array(ids,dtype = 'float64')
#     labels = np.array(labels)
## Spliting the dataset
#     train_x,train_y = pre_process_data(train_x,train_y)
#     valid_x,valid_y = pre_process_data(valid_x,valid_y)


In [12]:
labels_df = pd.read_csv(labels_path)
breed = labels_df["breed"].unique()
print("Number of Breed: ", len(breed))


Number of Breed:  11


In [13]:
breed2id = {name: i for i, name in enumerate(breed)}

In [14]:
ids = glob(train_path)
labels = []

for image_id in ids:
    image_id = image_id.split("\\")[-1].split(".")[0]
    breed_name = np.array(labels_df[labels_df.id == image_id]["breed"])[0]
    breed_idx = breed2id[breed_name]
    labels.append(breed_idx)


'Other_breeds/Not_Defined'

In [15]:
ids = ids[:1000]
labels = labels[:1000]

In [16]:
train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

In [17]:
size = 224
num_classes = 11
lr = 1e-4
batch = 16
epochs = 10

In [41]:
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

<RepeatDataset shapes: ((None, 224, 224, 3), (None, 11)), types: (tf.float32, tf.int32)>

In [53]:
callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-4)
]
train_steps = (len(train_x)//batch) + 1
valid_steps = (len(valid_x)//batch) + 1

In [54]:
main_model.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        epochs=25,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        callbacks=callbacks_list)

Epoch 1/25
51/51 [==============================] - 96s 2s/step - loss: 0.4650 - accuracy: 0.9093 - val_loss: 0.6271 - val_accuracy: 0.9150
Epoch 2/25
51/51 [==============================] - 97s 2s/step - loss: 0.4546 - accuracy: 0.9179 - val_loss: 0.5339 - val_accuracy: 0.9150
Epoch 3/25
51/51 [==============================] - 101s 2s/step - loss: 0.4412 - accuracy: 0.9179 - val_loss: 0.5011 - val_accuracy: 0.9150
Epoch 4/25
51/51 [==============================] - 101s 2s/step - loss: 0.4378 - accuracy: 0.9142 - val_loss: 0.4988 - val_accuracy: 0.9150
Epoch 5/25
51/51 [==============================] - 102s 2s/step - loss: 0.4140 - accuracy: 0.9203 - val_loss: 0.6357 - val_accuracy: 0.9150
Epoch 6/25
51/51 [==============================] - 101s 2s/step - loss: 0.4149 - accuracy: 0.9191 - val_loss: 0.7443 - val_accuracy: 0.9100
Epoch 7/25
51/51 [==============================] - 100s 2s/step - loss: 0.3936 - accuracy: 0.9203 - val_loss: 0.6460 - val_accuracy: 0.9000
Epoch 8/25
51/5

In [55]:
from tqdm import tqdm


# test_set = []
# test_set_ids = []
# for curImage in os.listdir('E:\\DAG_BREED_CLASSIFICATION\\test'):
#     test_set_ids.append(os.path.splitext(curImage)[0])
# #     print(test_set_ids)
# #     curImage = cv2.imread('E:\\DAG_BREED_CLASSIFICATION\\test/'+curImage)
# #     #print(curImage)
# #     #test_set.append(cv2.resize(curImage,(224,224)))
#     test_data = 
    
#
# breed2id = {name: i for i, name in enumerate(breed)}
# id2breed = {i: name for i, name in enumerate(breed)}

In [56]:
# for i, path in tqdm(enumerate(valid_x[:10])):
#     image = read_image(path, 224)
#     image = np.expand_dims(image, axis=0)
#     pred = model.predict(image)[0]
#     label_idx = np.argmax(pred)
#     breed_name = id2breed[label_idx]

#     ori_breed = id2breed[valid_y[i]]
#     ori_image = cv2.imread(path, cv2.IMREAD_COLOR)

#     ori_image = cv2.putText(ori_image, breed_name, (0, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
#     ori_image = cv2.putText(ori_image, ori_breed, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

#     cv2.imwrite(f"save/valid_{i}.png", ori_image)

In [ ]:
# path = "E:\DAG_BREED_CLASSIFICATION"
# train_path = os.path.join(path, "train/*")
# test_path = os.path.join(path, "test/*")
# labels_path = os.path.join(path, "labels.csv")

# labels_df = pd.read_csv(labels_path)
# breed = labels_df["breed"].unique()
# print("Number of Breed: ", len(breed))

#breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

# ids = glob(train_path)
# labels = []

# for image_id in ids:
#     image_id = image_id.split("/")[-1].split(".")[0]
#     breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
#     breed_idx = breed2id[breed_name]
#     labels.append(breed_idx)

# ids = ids[:1000]
# labels = labels[:1000]

## Spliting the dataset
# train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
# train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

## Model
#model = tf.keras.models.load_model("model.h5")
# idstest = glob(test_path)
# print(idstest)
pred = []
ori_breed = []
ori_image = []
for path in idstest:
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)    
    pred = main_model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]

    ori_breed = id2breed[valid_y[i]]
    ori_image = cv2.imread(path, cv2.IMREAD_COLOR)